In [1]:
! ollama pull gemma3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling aeda25e63ebd: 100% ▕██████████████████▏ 3.3 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling dd084c7d92a3: 100% ▕██████████████████▏ 8.4 KB                         
pulling 3116c5225075: 100% ▕██████████████████▏   77 B                         
pulling b6ae5839783f: 100% ▕██████████████████▏  489 B                         
verifying sha256 digest 
writing manifest 
success 


In [2]:
import pandas as pd
import gradio as gr
from langchain.schema import SystemMessage

In [3]:
from langchain_community.llms import Ollama
from langchain_experimental.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent
import openpyxl
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [4]:
llm = Ollama(model="gemma3", base_url="http://127.0.0.1:11434")

/var/folders/z1/2kj6d1ld1w5f46gd1x6cqh2m0000gn/T/ipykernel_2206/1999449518.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma3", base_url="http://127.0.0.1:11434")


In [5]:
system_prompt = """
You are a helpful assistant named Chitti that provides concise and accurate responses based on the provided dataframe.
If the customer greets with Hi or Hello, greet them and introduce yourself.
If the customer talks about anything general, respond based on their message.
Always be helpful and answer questions about the data in a clear manner.
"""

In [6]:
current_agent = None

In [7]:
def process_file(file):
    """Process the uploaded Excel file and return a message."""
    global current_agent
    
    if file is None:
        return "Please upload an Excel file to begin."
    
    try:
        # Read the Excel file
        df = pd.read_excel(file.name)
        
        # Create the agent with only supported parameters
        current_agent = create_pandas_dataframe_agent(
            llm=llm,
            df=df,
            verbose=False,
            handle_parsing_errors=True,
            prefix=system_prompt,allow_dangerous_code= True
        )
        
        # Return the dataframe information
        return f"✅ File loaded successfully!"
    except Exception as e:
        current_agent = None
        return f"Error processing file: {str(e)}"


In [8]:

def chat(message, history):
    """Handle chat interactions with the agent."""
    global current_agent
    
    if current_agent is None:
        return "Please upload an Excel file first before asking questions."
    
    try:
        messages = history + [{"role":"user", "content":message}]
        result = current_agent.invoke({"input": messages})
        return result['output']
    except Exception as e:
        x = f"Error processing your request: {str(e)}"
        y = x.split('Could not parse LLM output: ')[1]
        z = y.split('For troubleshooting, visit:')[0]
        message = message+z
        return z


In [9]:
with gr.Blocks(css="""
    .gradio-container, .gradio-container > div {
        height: 100vh !important;
    }
    .chat-column {
        height: calc(100vh - 140px);
    }
    .file-column {
        height: 50%;
    }
""") as demo:
    gr.Markdown("# 📊 Chat with Your Excel Data")
    
    with gr.Row():
        with gr.Column(scale=1, elem_classes="file-column"):
            file_input = gr.File(
                label="Upload Excel File",
                file_types=[".xlsx", ".xls"],
                type="filepath",
                scale=1,
                min_width=20
            )
            status_output = gr.Markdown("Please upload an Excel file to begin.")
        
        with gr.Column(scale=2, elem_classes="chat-column", show_progress = True):
            chatbot = gr.ChatInterface(
                fn=chat,
                fill_height=True, 
                fill_width=True,
                type="messages",
                stop_btn=True, 
                submit_btn=True
            )
    
    # Process file upload
    file_input.change(
        fn=process_file,
        inputs=[file_input],
        outputs=[status_output]
    )
    
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://bfb8a3d78ddf5283df.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/opt/anaconda3/envs/llms/lib/python3.11/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [39]:
# give me a summary of the data provided 

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(
